In [9]:
import csv
import time
import numpy as np
import copy
import math
import random
from IPython.display import clear_output
import time

In [10]:
# jupyter notebook --NotebookApp.iopub_msg_rate_limit=10000000

In [11]:

def list_to_dict(d):
    '''
    This function convert the input list of list to a dictionary,
    
    Inputs:
            d: list of lists of variables: e.g. [[1,2],[3,4],[1,5]]
            
    Outputs:
            d_dict = corresponding dictionary, e.g. {1:[2,5], 2:[1], 3:[4], 4[3], 5:[1]}
    '''
    d_dict = {}
    for i in d:
        try:
            d_dict[i[0]].append(i[1])
        except:
            d_dict[i[0]] = [i[1]]

        try:
            d_dict[i[1]].append(i[0])
        except:

            d_dict[i[1]] = [i[0]]
    return d_dict


def dict_to_list(d_dict):
    '''
    This function convert the input dictionary to a list of lists,
    
    Inputs:
            d_dict = corresponding dictionary, e.g. {1:[2,5], 2:[1], 3:[4], 4[3], 5:[1]}
            
    Outputs:
            d: list of lists of variables: e.g. [[1,2],[3,4],[1,5]]
    '''
    d_list = []
    for i in d_dict:
        for j in d_dict[i]:
            d_list.append([i,j])
            d_dict[j].remove(i)

    return d_list

In [38]:

def reduction_dictbase(my_dict):
    '''    
    This function reduced the input instances by removing the variables that are not
    changing the sign.
    
    Inputs: 
            d: input instances, list of lists of clasuse/variables,
            e.g. [[2,4], [-5,6], [7,2]] --> 3 clauses and 2,4,5,6,7 are variables
            
    Outputs:
            dd: reducded instances
    
    '''

    my_dict_old = copy.deepcopy(my_dict)
    for k in range(3000): # repeat reduction many times
        
        # early stop if there was no progress in the previous iteration
        if len(my_dict_old) == len(my_dict) and k!=0: 
            break
            
        my_dict = copy.deepcopy(my_dict_old)
        for i in my_dict:
            try: 
                check = my_dict_old[-i] # check if -i is also in the variabels
            except:
                # remove i from value of all keys that are in the key i value
                # i: [j,k,p] removes i from j:[....], k:[....], and p:[.....]
                for j in my_dict_old[i]:
                    my_dict_old[j].remove(i)
                
                # delet key i
                del my_dict_old[i]
        
        # delet keys which has [] values
        my_dict_old = {key: value  for key, value in my_dict_old.items() if value != []}
            
        
        clear_output(wait=True)
        print('Iteration number:')
        print(k+1)
        print('Size of the reduced list:')
        print(len(my_dict_old))
        
    
    return my_dict


In [39]:

def Papadimitriou(d):
    '''
    Papadimitriou algorithm for 2SAT problem
    
    Inputs: 
            d: input instances, list of lists of clauses/variables,
            e.g. [[2,4], [-5,6], [7,2]] --> 3 clauses and 2,4,5,6,7 are variables
            
    Outputs:
            satisfies: True/False for Satisfiable/non-Satisfiable 
            satisfiable_instance: an intance that satisfies the probelm, if it is True (Satisfiable) other wise it is empty
    '''
    
    satisfies = False

    to_make_unique = [item for sublist in d for item in sublist] # unique variables in d
    n = len(set(to_make_unique)) # n is the number of variables in d
    
    # if reducded to zero clause, means that it is satisfable
    if n == 0:
        satisfies = True
        satisfiable_instance = {}
        
        return satisfies, satisfiable_instance
        
        
    m = len(d) # m is the number of clauses in d

    to_make_unique = [item for sublist in d for item in sublist if item>0] # only positives
    unique_variables = list(set(to_make_unique))

    # repeat log2(n) times
    for k in range(0,int(math.log(n,2))+2):
        
        # too break out of the outer loop, done if satisfies is True (for all clauses)
        if satisfies == True:
            break
            
        # random intitial assignment
        a = {}
        a = {i: random.choice([1, -1]) for i in unique_variables} # a disctionary with variables (only positives) as key and random 1 (True) and -1 (False) as values

        for j in range(0, (2*(n**2)) + 1): # repeat 2n^2 times
            # check if current assigment satisfies all clasuse
            # going through all clauses
            satisfies = True
            for i in range(0,m):
                first_sign = (d[i][0]/abs(d[i][0])) # sign of the first variable in the clause
                second_sign = (d[i][1]/abs(d[i][1])) # sign of the second variable in the clause
                first = a[abs(d[i][0])] * first_sign # sign of the first variable in the clause (if >0 means TRUE if <0 means FALSE)
                second = a[abs(d[i][1])] * second_sign # sign of the second variable in the clause (if >0 means TRUE if <0 means FALSE)

                if first < 0 and second < 0:
                    satisfies = False
                    # flip one of the variables in the clause
                    variable = random.choice([1, 2]) # choose randomely variable 1 or 2
                    if variable == 1:
                        a[abs(d[i][0])] *= -1 # flipping the first variable
                    else:
                        a[abs(d[i][1])] *= -1 # flipping the second variable
                    break

            # too break out of the inner loop, done if satisfies is True (for all clauses)
            if satisfies == True:
                break

    if satisfies == True:
        satisfiable_instance = a
    else:
        satisfiable_instance = {}
        
        
    return satisfies, satisfiable_instance

In [40]:
# change d into dictionary
# reading the input file as a list
with open('2sat1.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
143
Size of the reduced list:
12
__________________________________________ 

Size of the final reduced list:
6
--- 1.6988530158996582 seconds ---
__________________________________________ 

Is the instance satisfiable?  

True
--- 0.000997304916381836 seconds ---


In [30]:
# change d into dictionary
# reading the input file as a list
with open('2sat2.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
89
Size of the reduced list:
108
__________________________________________ 

Size of the final reduced list:
57
--- 3.2282943725585938 seconds ---
__________________________________________ 

Is the instance satisfiable?  

False
--- 1.6864843368530273 seconds ---


In [31]:
# change d into dictionary
# reading the input file as a list
with open('2sat3.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
166
Size of the reduced list:
576
__________________________________________ 

Size of the final reduced list:
295
--- 6.79911208152771 seconds ---
__________________________________________ 

Is the instance satisfiable?  

True
--- 2.0591118335723877 seconds ---


In [32]:
# change d into dictionary
# reading the input file as a list
with open('2sat4.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
130
Size of the reduced list:
22
__________________________________________ 

Size of the final reduced list:
11
--- 10.43192172050476 seconds ---
__________________________________________ 

Is the instance satisfiable?  

True
--- 0.0 seconds ---


In [33]:
# change d into dictionary
# reading the input file as a list
with open('2sat5.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
144
Size of the reduced list:
196
__________________________________________ 

Size of the final reduced list:
101
--- 14.061365842819214 seconds ---
__________________________________________ 

Is the instance satisfiable?  

False
--- 12.526309728622437 seconds ---


In [41]:
# change d into dictionary
# reading the input file as a list
with open('2sat6.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    d = list(reader)

n  = int(d[0][0]) # n is the number of variables
d.pop(0)

d = [[int(i[0]),int(i[1])] for i in d] # convert d into integers
m = len(d) # m is the number of clauses


start_time = time.time()
d_dict = list_to_dict(d)
dd_dict = reduction_dictbase(d_dict)
# {key: value  for key, value in dd1.items() if value != []}
print('__________________________________________ \n')
print('Size of the final reduced list:')
dd= dict_to_list(dd_dict)
print(len(dd))
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
check, assignment = Papadimitriou(dd)
print('__________________________________________ \n')
print('Is the instance satisfiable?  \n')
print(check)
# print('\n If Ture, an assignment example that satisfies it (the rest of variables could be anything, True(1) or False(-1))')
# print(assignment)
print("--- %s seconds ---" % (time.time() - start_time))

Iteration number:
159
Size of the reduced list:
50
__________________________________________ 

Size of the final reduced list:
26
--- 17.060615301132202 seconds ---
__________________________________________ 

Is the instance satisfiable?  

False
--- 0.19051098823547363 seconds ---
